# Assignment: Cluster Analysis

In [1]:
from datetime import datetime
print(f'Updated {datetime.now().strftime("%d.%m.%Y")} / Tatu Erkinjuntti')

Updated 17.11.2025 / Tatu Erkinjuntti


## Learning Objectives
This assignment introduces basics of clustering analysis in unsupervised machine learning implemented with Python programming language. In doing so, you will also learn how to present information clearly and how to document and explain calculation steps created in notebooks.

## Initial information
IT Department in your company has circulated the following dataset across the company. A new way of collecting data has been put into operation but nobody seems to know what to do with data collected. One day the dataset is in your mailbox.

Perform a cluster analysis to the dataset given below. You can also use your own dataset, if you so wish.

Analyse the results carefully and give marketing department instructions how to exploit them in their work.

In [2]:
# Import libraries
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
# Define data location and import data as a data frame.
file_path = "../data/sales_data_sample.csv"
# NOTE: .csv file is not formated in UTF-8...  ISO-8859-1 encoding seems to work.
df = pd.read_csv(file_path, encoding='ISO-8859-1')
# Check data (first five rows).
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


It seems that we have data on Motorcycle sales. Lets check the PRODUCTLINE column to be sure.

In [4]:
df['PRODUCTLINE'].unique()

array(['Motorcycles', 'Classic Cars', 'Trucks and Buses', 'Vintage Cars',
       'Planes', 'Ships', 'Trains'], dtype=object)

OK, it seems that we have other products than just motorcycles. Lets check for specifics.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2823 non-null   int64  
 1   QUANTITYORDERED   2823 non-null   int64  
 2   PRICEEACH         2823 non-null   float64
 3   ORDERLINENUMBER   2823 non-null   int64  
 4   SALES             2823 non-null   float64
 5   ORDERDATE         2823 non-null   object 
 6   STATUS            2823 non-null   object 
 7   QTR_ID            2823 non-null   int64  
 8   MONTH_ID          2823 non-null   int64  
 9   YEAR_ID           2823 non-null   int64  
 10  PRODUCTLINE       2823 non-null   object 
 11  MSRP              2823 non-null   int64  
 12  PRODUCTCODE       2823 non-null   object 
 13  CUSTOMERNAME      2823 non-null   object 
 14  PHONE             2823 non-null   object 
 15  ADDRESSLINE1      2823 non-null   object 
 16  ADDRESSLINE2      302 non-null    object 


Lets check for NULL values in an accending order.

In [6]:
df.isnull().sum().sort_values(ascending=False)

ADDRESSLINE2        2521
STATE               1486
TERRITORY           1074
POSTALCODE            76
ORDERNUMBER            0
CUSTOMERNAME           0
CONTACTFIRSTNAME       0
CONTACTLASTNAME        0
COUNTRY                0
CITY                   0
ADDRESSLINE1           0
PHONE                  0
PRODUCTCODE            0
QUANTITYORDERED        0
MSRP                   0
PRODUCTLINE            0
YEAR_ID                0
MONTH_ID               0
QTR_ID                 0
STATUS                 0
ORDERDATE              0
SALES                  0
ORDERLINENUMBER        0
PRICEEACH              0
DEALSIZE               0
dtype: int64

It seems that there are columns that contain quite alot of missing values. Since we are dealing with attributes that might not be used in a destination country, my assessment is that for Cluster analysis it is safe to just remove them.

In [7]:
df.drop(columns=['ADDRESSLINE2','STATE','TERRITORY', 'POSTALCODE'], inplace=True)
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,CITY,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,95,S10_1678,Land of Toys Inc.,2125557818,897 Long Airport Avenue,NYC,USA,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,95,S10_1678,Reims Collectables,26.47.1555,59 rue de l'Abbaye,Reims,France,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,95,S10_1678,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,France,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,95,S10_1678,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,Pasadena,USA,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,95,S10_1678,Corporate Gift Ideas Co.,6505551386,7734 Strong St.,San Francisco,USA,Brown,Julie,Medium


Before we start selecting attributes for cluster analysis,we might want to consider what are we trying to answer and to who? Since are target group is the marketing department, we might want to know what country consumes most products.
